In [87]:
# Importing libraries
import pandas as pd
import re
import sqlite3
import requests
import os

**Extract**

In [88]:
# Base API URL
BASE_URL = "https://data.gov.uk/api/action/"

def package_search(query="price paid"):
    url = f"{BASE_URL}package_search"
    params = {"q": query, "rows": 5}
    response = requests.get(url, params=params)
    response.raise_for_status()

    datasets = response.json().get("result", {}).get("results", [])
    price_paid_resources = []  # To collect relevant resource URLs

    for dataset in datasets:
        # Filter datasets with "price paid" in their title or tags
        if "price paid" in dataset.get("title", "").lower() or \
           any("price paid" in tag.lower() for tag in dataset.get("tags", [])):

            # Fetch resources for the filtered dataset
            resources = dataset.get("resources", [])
            for resource in resources:
                resource_url = resource.get("url")
                if resource_url and resource.get("format", "").lower() == "csv":

                    price_paid_resources.append(resource_url)

    return price_paid_resources

def download_csv(resource_url, save_path="downloaded.csv"):
    print(f"Downloading CSV from: {resource_url}")
    response = requests.get(resource_url, stream=True)
    response.raise_for_status()  # Ensure the request was successful

    with open(save_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
    print(f"File downloaded successfully and saved as: {save_path}")

# Run the search and download the CSV
price_paid_urls = package_search()
if price_paid_urls:
    for index, url in enumerate(price_paid_urls, start=1):
        save_path = f"price_paid_dataset_{index}.csv"
        download_csv(url, save_path)
else:
    print("No Price Paid Datasets found.")

File downloaded successfully and saved as: price_paid_dataset_1.csv
File downloaded successfully and saved as: price_paid_dataset_2.csv
File downloaded successfully and saved as: price_paid_dataset_3.csv
File downloaded successfully and saved as: price_paid_dataset_4.csv
File downloaded successfully and saved as: price_paid_dataset_5.csv
File downloaded successfully and saved as: price_paid_dataset_6.csv
File downloaded successfully and saved as: price_paid_dataset_7.csv
File downloaded successfully and saved as: price_paid_dataset_8.csv
File downloaded successfully and saved as: price_paid_dataset_9.csv
File downloaded successfully and saved as: price_paid_dataset_10.csv
File downloaded successfully and saved as: price_paid_dataset_11.csv
File downloaded successfully and saved as: price_paid_dataset_12.csv
File downloaded successfully and saved as: price_paid_dataset_13.csv
File downloaded successfully and saved as: price_paid_dataset_14.csv
File downloaded successfully and saved as: 

**Transform**

In [114]:
# Reading dataset
data = pd.read_csv("pp-2017-part1.csv")

In [115]:
# Adding coulumn names
data.columns = ['TransactionId', 'Price', 'DateofTransfer', 'Postcode', 'PropertyType', 'PropertyAge', 'Tenure','PAON','SAON', 'Street', 'Locality','Town/City',"District",'County','PPDCategory','Record_Status']
data

,TransactionId,Price,DateofTransfer,Postcode,PropertyType,PropertyAge,Tenure,PAON,SAON,Street,Locality,Town/City,District,County,PPDCategory,Record_Status
0,{50F18103-65BF-9FD5-E050-A8C063054923},195000,2017-04-11 00:00,YO8 9GN,S,N,F,5,NaN,ST MARYS COURT,HAMBLETON,SELBY,SELBY,NORTH YORKSHIRE,A,A
1,{50F18103-65C0-9FD5-E050-A8C063054923},275000,2017-05-05 00:00,DL8 4BD,D,N,F,LYNDHURST,NaN,NaN,CARLTON,LEYBURN,RICHMONDSHIRE,NORTH YORKSHIRE,A,A
2,{50F18103-65C1-9FD5-E050-A8C063054923},715000,2017-04-28 00:00,HG5 9HA,D,N,F,GATESHILL MANOR,NaN,RIPLEY ROAD,NaN,KNARESBOROUGH,HARROGATE,NORTH YORKSHIRE,A,A
3,{50F18103-65C2-9FD5-E050-A8C063054923},167500,2017-04-03 00:00,YO12 6TW,S,N,F,39,NaN,HIRSTEAD ROAD,NaN,SCARBOROUGH,SCARBOROUGH,NORTH YORKSHIRE,A,A
4,{50F18103-65C4-9FD5-E050-A8C063054923},300000,2017-05-22 00:00,YO18 8QA,D,N,F,CORNER COTTAGE,NaN,NaN,NEWTON ON RAWCLIFFE,PICKERING,RYEDALE,NORTH YORKSHIRE,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533465,{49B7852A-7C05-7921-E050-A8C063056E8D},275000,2017-02-17 00:00,CR7 7BP,T,N,F,19,NaN,ECCLESBOURNE ROAD,NaN,THORNTON HEATH,CROYDON,GREATER LONDON,A,A
533466,{49B7852A-7C06-7921-E050-A8C063056E8D},900000,2017-02-14 00:00,KT22 9QR,D,N,F,8,NaN,GATESDEN ROAD,FETCHAM,LEATHERHEAD,MOLE VALLEY,SURREY,A,A
533467,{49B7852A-7C07-7921-E050-A8C063056E8D},660000,2017-01-10 00:00,KT6 5JP,S,N,F,31,NaN,FLEECE ROAD,LONG DITTON,SURBITON,ELMBRIDGE,SURREY,A,A
533468,{49B7852A-7C08-7921-E050-A8C063056E8D},385000,2017-02-17 00:00,KT4 8NP,T,N,F,200,NaN,BUCKLAND WAY,NaN,WORCESTER PARK,SUTTON,GREATER LONDON,A,A


In [116]:
# More information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533470 entries, 0 to 533469
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   TransactionId   533470 non-null  object
 1   Price           533470 non-null  int64 
 2   DateofTransfer  533470 non-null  object
 3   Postcode        530499 non-null  object
 4   PropertyType    533470 non-null  object
 5   PropertyAge     533470 non-null  object
 6   Tenure          533470 non-null  object
 7   PAON            533470 non-null  object
 8   SAON            79066 non-null   object
 9   Street          523214 non-null  object
 10  Locality        200100 non-null  object
 11  Town/City       533470 non-null  object
 12  District        533470 non-null  object
 13  County          533470 non-null  object
 14  PPDCategory     533470 non-null  object
 15  Record_Status   533470 non-null  object
dtypes: int64(1), object(15)
memory usage: 65.1+ MB


In [117]:
# Checking for null values
data.isnull().sum()

,0
TransactionId,0
Price,0
DateofTransfer,0
Postcode,2971
PropertyType,0
PropertyAge,0
Tenure,0
PAON,0
SAON,454404
Street,10256


In [118]:
# Drop all the Postcodes with null values
data.dropna(subset=["Postcode"], inplace=True)
# Checking for Null Values again
data.isnull().sum()

,0
TransactionId,0
Price,0
DateofTransfer,0
Postcode,0
PropertyType,0
PropertyAge,0
Tenure,0
PAON,0
SAON,452653
Street,9759


In [119]:
# Delete column district
data.drop(columns=["District"], inplace=True)
# Show the names of columns of dataset
data.columns

Index(['TransactionId', 'Price', 'DateofTransfer', 'Postcode', 'PropertyType',
       'PropertyAge', 'Tenure', 'PAON', 'SAON', 'Street', 'Locality',
       'Town/City', 'County', 'PPDCategory', 'Record_Status'],
      dtype='object')

In [120]:
# Standardize address

# Function to standardize the address and its length
def standardize_address(row):
    # Ensure all columns exist and handle NaN values
    address_components = [
        str(row.get("PAON", "N/A")) if pd.notna(row.get("PAON")) else "N/A",
        str(row.get("SAON", "N/A")) if pd.notna(row.get("SAON")) else "N/A",
        str(row.get("Street", "N/A")) if pd.notna(row.get("Street")) else "N/A",
        str(row.get("Locality", "N/A")) if pd.notna(row.get("Locality")) else "N/A",
        str(row.get("Town/City", "N/A")) if pd.notna(row.get("Town/City")) else "N/A",
        str(row.get("County", "N/A")) if pd.notna(row.get("County")) else "N/A",
        str(row.get("Postcode", "N/A")) if pd.notna(row.get("Postcode")) else "N/A"
    ]

    # Join components with commas
    address = ", ".join(address_components)

    # Convert to title case for consistency
    address = address.title()

    # Clean up any double spaces and trim
    address = re.sub(r"\s{2,}", " ", address).strip()

    return address

# Apply the standardize_address function to the DataFrame
data["StandardizedAddress"] = data.apply(standardize_address, axis=1)
data.head(10)

,TransactionId,Price,DateofTransfer,Postcode,PropertyType,PropertyAge,Tenure,PAON,SAON,Street,Locality,Town/City,County,PPDCategory,Record_Status,StandardizedAddress
0,{50F18103-65BF-9FD5-E050-A8C063054923},195000,2017-04-11 00:00,YO8 9GN,S,N,F,5,NaN,ST MARYS COURT,HAMBLETON,SELBY,NORTH YORKSHIRE,A,A,"5, N/A, St Marys Court, Hambleton, Selby, Nort..."
1,{50F18103-65C0-9FD5-E050-A8C063054923},275000,2017-05-05 00:00,DL8 4BD,D,N,F,LYNDHURST,NaN,NaN,CARLTON,LEYBURN,NORTH YORKSHIRE,A,A,"Lyndhurst, N/A, N/A, Carlton, Leyburn, North Y..."
2,{50F18103-65C1-9FD5-E050-A8C063054923},715000,2017-04-28 00:00,HG5 9HA,D,N,F,GATESHILL MANOR,NaN,RIPLEY ROAD,NaN,KNARESBOROUGH,NORTH YORKSHIRE,A,A,"Gateshill Manor, N/A, Ripley Road, N/A, Knares..."
3,{50F18103-65C2-9FD5-E050-A8C063054923},167500,2017-04-03 00:00,YO12 6TW,S,N,F,39,NaN,HIRSTEAD ROAD,NaN,SCARBOROUGH,NORTH YORKSHIRE,A,A,"39, N/A, Hirstead Road, N/A, Scarborough, Nort..."
4,{50F18103-65C4-9FD5-E050-A8C063054923},300000,2017-05-22 00:00,YO18 8QA,D,N,F,CORNER COTTAGE,NaN,NaN,NEWTON ON RAWCLIFFE,PICKERING,NORTH YORKSHIRE,A,A,"Corner Cottage, N/A, N/A, Newton On Rawcliffe,..."
5,{50F18103-65C5-9FD5-E050-A8C063054923},304000,2017-05-12 00:00,YO10 4HN,S,N,F,73,NaN,HESLINGTON LANE,NaN,YORK,YORK,A,A,"73, N/A, Heslington Lane, N/A, York, York, Yo1..."
6,{50F18103-65C6-9FD5-E050-A8C063054923},782000,2017-05-19 00:00,YO30 7DZ,D,N,F,THE OLD STABLES,NaN,GALMANHOE LANE,NaN,YORK,YORK,A,A,"The Old Stables, N/A, Galmanhoe Lane, N/A, Yor..."
7,{50F18103-65C7-9FD5-E050-A8C063054923},315000,2017-04-28 00:00,BD23 6QN,D,N,F,24,NaN,BRACKENLEY DRIVE,EMBSAY,SKIPTON,NORTH YORKSHIRE,A,A,"24, N/A, Brackenley Drive, Embsay, Skipton, No..."
8,{50F18103-65C8-9FD5-E050-A8C063054923},250000,2017-04-20 00:00,YO13 0PY,T,N,F,19,NaN,THE PARK,SCALBY,SCARBOROUGH,NORTH YORKSHIRE,A,A,"19, N/A, The Park, Scalby, Scarborough, North ..."
9,{50F18103-65C9-9FD5-E050-A8C063054923},595000,2017-04-19 00:00,YO13 0QR,D,N,F,THE OLD NURSERIES,3,SOUTH STREET,SCALBY,SCARBOROUGH,NORTH YORKSHIRE,A,A,"The Old Nurseries, 3, South Street, Scalby, Sc..."


In [121]:
# Duplicates in the dataset
data.duplicated().sum()

0

In [122]:
date_dim = data[["TransactionId", "DateofTransfer"]]
# Convert to datetime format
date_dim["DateofTransfer"] = pd.to_datetime(date_dim["DateofTransfer"])
# Extract day, month, and year
date_dim["Day"] = date_dim["DateofTransfer"].dt.day
date_dim["Month"] = date_dim["DateofTransfer"].dt.month
date_dim["Year"] = date_dim["DateofTransfer"].dt.year
date_dim["Quarter"] = date_dim["DateofTransfer"].dt.quarter
date_dim["DateId"] = date_dim.index + 1
date_dim


<ipython-input-122-db07adff9e27>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_dim["DateofTransfer"] = pd.to_datetime(date_dim["DateofTransfer"])
<ipython-input-122-db07adff9e27>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_dim["Day"] = date_dim["DateofTransfer"].dt.day
<ipython-input-122-db07adff9e27>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,TransactionId,DateofTransfer,Day,Month,Year,Quarter,DateId
0,{50F18103-65BF-9FD5-E050-A8C063054923},2017-04-11,11,4,2017,2,1
1,{50F18103-65C0-9FD5-E050-A8C063054923},2017-05-05,5,5,2017,2,2
2,{50F18103-65C1-9FD5-E050-A8C063054923},2017-04-28,28,4,2017,2,3
3,{50F18103-65C2-9FD5-E050-A8C063054923},2017-04-03,3,4,2017,2,4
4,{50F18103-65C4-9FD5-E050-A8C063054923},2017-05-22,22,5,2017,2,5
...,...,...,...,...,...,...,...
533465,{49B7852A-7C05-7921-E050-A8C063056E8D},2017-02-17,17,2,2017,1,533466
533466,{49B7852A-7C06-7921-E050-A8C063056E8D},2017-02-14,14,2,2017,1,533467
533467,{49B7852A-7C07-7921-E050-A8C063056E8D},2017-01-10,10,1,2017,1,533468
533468,{49B7852A-7C08-7921-E050-A8C063056E8D},2017-02-17,17,2,2017,1,533469


In [123]:
# Create Dimension Table Property Type
property_type_dim = data[["TransactionId","PropertyType", "PropertyAge", "Tenure"]]
property_type_dim["PropertyTypeId"] = property_type_dim.index + 1
property_type_dim

<ipython-input-123-a030d4660eae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  property_type_dim["PropertyTypeId"] = property_type_dim.index + 1


,TransactionId,PropertyType,PropertyAge,Tenure,PropertyTypeId
0,{50F18103-65BF-9FD5-E050-A8C063054923},S,N,F,1
1,{50F18103-65C0-9FD5-E050-A8C063054923},D,N,F,2
2,{50F18103-65C1-9FD5-E050-A8C063054923},D,N,F,3
3,{50F18103-65C2-9FD5-E050-A8C063054923},S,N,F,4
4,{50F18103-65C4-9FD5-E050-A8C063054923},D,N,F,5
...,...,...,...,...,...
533465,{49B7852A-7C05-7921-E050-A8C063056E8D},T,N,F,533466
533466,{49B7852A-7C06-7921-E050-A8C063056E8D},D,N,F,533467
533467,{49B7852A-7C07-7921-E050-A8C063056E8D},S,N,F,533468
533468,{49B7852A-7C08-7921-E050-A8C063056E8D},T,N,F,533469


In [124]:
address_dim = data[["TransactionId","StandardizedAddress"]]
# Split the StandardizedAddress
split_address = address_dim["StandardizedAddress"].str.split(", ", expand=True, n=6)
# Rename the columns
split_address.columns = ["PAON", "SAON", "Street", "Locality", "Town_City", "County", "Postcode"]
address_dim = pd.concat([address_dim["TransactionId"], split_address], axis=1)
address_dim["AddressId"] = address_dim.index + 1
address_dim.head()

,TransactionId,PAON,SAON,Street,Locality,Town_City,County,Postcode,AddressId
0,{50F18103-65BF-9FD5-E050-A8C063054923},5,N/A,St Marys Court,Hambleton,Selby,North Yorkshire,Yo8 9Gn,1
1,{50F18103-65C0-9FD5-E050-A8C063054923},Lyndhurst,N/A,N/A,Carlton,Leyburn,North Yorkshire,Dl8 4Bd,2
2,{50F18103-65C1-9FD5-E050-A8C063054923},Gateshill Manor,N/A,Ripley Road,N/A,Knaresborough,North Yorkshire,Hg5 9Ha,3
3,{50F18103-65C2-9FD5-E050-A8C063054923},39,N/A,Hirstead Road,N/A,Scarborough,North Yorkshire,Yo12 6Tw,4
4,{50F18103-65C4-9FD5-E050-A8C063054923},Corner Cottage,N/A,N/A,Newton On Rawcliffe,Pickering,North Yorkshire,Yo18 8Qa,5


In [125]:
# Create Dimension Table PPD Category
ppd_category_dim = data[["TransactionId","PPDCategory"]]
ppd_category_dim["PPDCategoryId"] = ppd_category_dim.index + 1
ppd_category_dim

<ipython-input-125-fd21dd2a01c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppd_category_dim["PPDCategoryId"] = ppd_category_dim.index + 1


,TransactionId,PPDCategory,PPDCategoryId
0,{50F18103-65BF-9FD5-E050-A8C063054923},A,1
1,{50F18103-65C0-9FD5-E050-A8C063054923},A,2
2,{50F18103-65C1-9FD5-E050-A8C063054923},A,3
3,{50F18103-65C2-9FD5-E050-A8C063054923},A,4
4,{50F18103-65C4-9FD5-E050-A8C063054923},A,5
...,...,...,...
533465,{49B7852A-7C05-7921-E050-A8C063056E8D},A,533466
533466,{49B7852A-7C06-7921-E050-A8C063056E8D},A,533467
533467,{49B7852A-7C07-7921-E050-A8C063056E8D},A,533468
533468,{49B7852A-7C08-7921-E050-A8C063056E8D},A,533469


In [126]:
# Create fact table with essential columns
fact_table = data[["TransactionId", "Price"]]

# Merge dimensions into the fact table
merged_table = (
    fact_table
    .merge(address_dim[["TransactionId", "AddressId"]], on="TransactionId", how="inner")
    .merge(date_dim[["TransactionId", "DateId"]], on="TransactionId", how="inner")
    .merge(property_type_dim[["TransactionId", "PropertyTypeId"]], on="TransactionId", how="inner")
    .merge(ppd_category_dim[["TransactionId", "PPDCategoryId"]], on="TransactionId", how="inner")
)
# Ensure the fact table has only the required columns
fact_table_final = merged_table[["TransactionId", "Price", "DateId", "PropertyTypeId", "AddressId", "PPDCategoryId"]]
fact_table_final

,TransactionId,Price,DateId,PropertyTypeId,AddressId,PPDCategoryId
0,{50F18103-65BF-9FD5-E050-A8C063054923},195000,1,1,1,1
1,{50F18103-65C0-9FD5-E050-A8C063054923},275000,2,2,2,2
2,{50F18103-65C1-9FD5-E050-A8C063054923},715000,3,3,3,3
3,{50F18103-65C2-9FD5-E050-A8C063054923},167500,4,4,4,4
4,{50F18103-65C4-9FD5-E050-A8C063054923},300000,5,5,5,5
...,...,...,...,...,...,...
530494,{49B7852A-7C05-7921-E050-A8C063056E8D},275000,533466,533466,533466,533466
530495,{49B7852A-7C06-7921-E050-A8C063056E8D},900000,533467,533467,533467,533467
530496,{49B7852A-7C07-7921-E050-A8C063056E8D},660000,533468,533468,533468,533468
530497,{49B7852A-7C08-7921-E050-A8C063056E8D},385000,533469,533469,533469,533469


**Load**

In [136]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('price_paid_data.db')
cursor = conn.cursor()

In [137]:
# Creating Table DimensionAddress
cursor.execute("""
CREATE TABLE IF NOT EXISTS DimensionAddress (
    TransactionId TEXT,
    AddressId INTEGER PRIMARY KEY AUTOINCREMENT,
    PAON TEXT,
    SAON TEXT,
    Street TEXT,
    Locality TEXT,
    Town_City TEXT,
    District TEXT,
    County TEXT,
    Postcode TEXT
);
""")

In [138]:
# Mapping to Address Schema
address_dim.to_sql(
    name='DimensionAddress',  # Name of the table
    con=conn,                 # Database connection
    if_exists='append',       # Options: 'fail', 'replace', 'append'
    index=False               # Do not write DataFrame index as a column
)


530499

In [139]:
# Creating Table DimensionDate
cursor.execute("""
CREATE TABLE IF NOT EXISTS DimensionDate (
    DateId INTEGER PRIMARY KEY AUTOINCREMENT,
    TransactionId TEXT,
    DateofTransfer DATE NOT NULL,
    Year INTEGER,
    Month INTEGER,
    Day INTEGER,
    Quarter INTEGER
);
""")

In [140]:
# Mapping to DimensionDate
date_dim.to_sql(
    name='DimensionDate',  # Name of the table
    con=conn,                 # Database connection
    if_exists='append',       # Options: 'fail', 'replace', 'append'
    index=False)

530499

In [141]:
# Creating Table DimensionPropertyType
cursor.execute("""
CREATE TABLE IF NOT EXISTS DimensionPropertyType (
    PropertyTypeId INTEGER PRIMARY KEY AUTOINCREMENT,
    TransactionId TEXT,
    PropertyAge TEXT,
    Tenure TEXT,
    PropertyType TEXT NOT NULL
);
""")

In [142]:
# Mapping to DimensionPropertyType
property_type_dim.to_sql(
    name='DimensionPropertyType',  # Name of the table
    con=conn,                 # Database connection
    if_exists='append',       # Options: 'fail', 'replace', 'append'
    index=False)

530499

In [143]:
# Creating Table DimensionPPDCategory
cursor.execute("""
CREATE TABLE IF NOT EXISTS DimensionPPDCategory (
    PPDCategoryId INTEGER PRIMARY KEY AUTOINCREMENT,
    TransactionId TEXT,
    PPDCategory TEXT NOT NULL
);
""")

In [144]:
# Mapping to DimensionPPDCategory
ppd_category_dim.to_sql(
    name='DimensionPPDCategory',  # Name of the table
    con=conn,                 # Database connection
    if_exists='append',       # Options: 'fail', 'replace', 'append'
    index=False
 )

530499

In [145]:
# Create Fact Table PropertyTransactions
cursor.execute("""
CREATE TABLE IF NOT EXISTS FactPropertyTransactions (
    TransactionId TEXT PRIMARY KEY,
    Price INTEGER NOT NULL,
    DateId INTEGER NOT NULL,
    PropertyTypeId INTEGER NOT NULL,
    AddressId INTEGER NOT NULL,
    PPDCategoryId INTEGER NOT NULL,
    FOREIGN KEY (PropertyTypeId) REFERENCES DimensionPropertyType(PropertyTypeId),
    FOREIGN KEY (DateId) REFERENCES DimensionDate(DateId),
    FOREIGN KEY (AddressId) REFERENCES DimensionAddress(AddressId),
    FOREIGN KEY (PPDCategoryId) REFERENCES DimensionPPDCategory(PPDCategoryId)
);
""")

In [146]:
fact_table_final.to_sql(
    name="FactPropertyTransactions",  # Name of the fact table
    con=conn,                         # Database connection
    if_exists="append",               # Append data to existing table
    index=False                       # Do not write DataFrame index as a column
)

530499

In [147]:
#Average Property Prices by City
print("Average Property Prices by City:")
cursor.execute("""
SELECT d.Town_City, AVG(f.Price) AS AvgPrice
FROM FactPropertyTransactions f
JOIN DimensionAddress d ON f.AddressId = d.AddressId
GROUP BY d.Town_City;
""")
average_prices = cursor.fetchall()
for row in average_prices:
    print(f"City: {row[0]}, Average Price: {row[1]:,.2f}")

Average Property Prices by City:
City: 19Th Floor, Average Price: 5,000.00
City: 27, Average Price: 15,750.00
City: Ab Kettleby, Average Price: 365,000.00
City: Abbeydale, Average Price: 229,000.00
City: Abbeymead, Average Price: 192,500.00
City: Abbots Langley, Average Price: 424,735.90
City: Abbots Leigh, Average Price: 1,330,000.00
City: Abbotts Ann, Average Price: 503,750.00
City: Aberaeron, Average Price: 230,136.36
City: Aberbargoed, Average Price: 80,000.00
City: Abercrave, Average Price: 190,000.00
City: Aberdare, Average Price: 111,532.25
City: Aberdovey, Average Price: 210,571.43
City: Abergavenny, Average Price: 266,548.10
City: Abergele, Average Price: 182,465.57
City: Aberporth, Average Price: 130,000.00
City: Abertillery, Average Price: 72,638.39
City: Aberystwyth, Average Price: 228,429.07
City: Abingdon, Average Price: 510,453.45
City: Acaster Malbis, Average Price: 435,000.00
City: Accrington, Average Price: 132,292.02
City: Acle, Average Price: 300,000.00
City: Acocks

In [148]:
# Export Average Property Prices to CSV
average_prices_df = pd.DataFrame(average_prices, columns=["City", "AveragePrice"])
average_prices_df.to_csv('average_property_prices_by_city.csv', index=False)